In [1]:
import os
from dotenv import load_dotenv
from autogen import UserProxyAgent, ConversableAgent, AssistantAgent, GroupChat, GroupChatManager

load_dotenv()

llm_config={"model": "gpt-4o-mini"}

# Project manager talk to engineers
project_manager_message = """
I have received the following message from a customer 
" I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.", 
Now I will talk to each engineers and please everyone give me your estimate plan for this project.

Each engineer must provide estimates in this EXACT format with ASCII table borders:

| Category                    | Amount of work      | Productivity Rate   |
|-----------------------------|---------------------|---------------------|
| [Task name]                 | [Amount]            | [Rate]              |

IMPORTANT: Each engineer must ONLY work on their specific domain and use the exact table format shown above.
Estimate based on the insurance recommendation app requirements provided.
"""

# Agent definition
customer = UserProxyAgent(
    name="customer",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False,
)

project_manager = AssistantAgent(
    name="project_manager",
    llm_config=llm_config,
    system_message=(
        "You are a project manager. Your task is to receive the customer's request, "
        "and send the message to your teammates. "
        "You do NOT need to analyze or expand it yourself. Focus on delivering the message for further planning."
    )
)

requirement_engineer = AssistantAgent(
    name="requirement_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a requirement engineer. You ONLY work on REQUIREMENTS tasks. "
        "Based on the insurance recommendation app project, estimate the requirements work needed. "
        "Provide your response in this EXACT format: "
        "**Requirement Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write requirements          | [your estimate]     | [your rate]         | "
        "| Review requirements         |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: user input forms, recommendation logic requirements, explanation features, mobile responsiveness requirements. "
        "Use realistic estimates based on project complexity."
    )
)

system_engineer = AssistantAgent(
    name="system_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a system engineer. You ONLY work on DESIGN DOCUMENT tasks. "
        "Based on the insurance recommendation app project, estimate the design documentation work needed. "
        "Provide your response in this EXACT format: "
        "**System Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write Design Document (DD)  | [your estimate]     | [your rate]         | "
        "| Review DD                   |                     |                     | "
        "|   Preparation for DD        | [your estimate]     | [your rate]         | "
        "|   Review DD                 | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: system architecture, database design, API design, recommendation algorithm design, security architecture. "
        "Use realistic estimates based on project complexity."
    )
)

software_engineer = AssistantAgent(
    name="software_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a software engineer. You ONLY work on PROGRAMMING/CODING tasks. "
        "Based on the insurance recommendation app project, estimate the programming work needed. "
        "Provide your response in this EXACT format: "
        "**Software Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write code                  | [your estimate]     | [your rate]         | "
        "| Unit Testing                |                     |                     | "
        "|   Prepare/Execute Test Cases| [your estimate]     | [your rate]         | "
        "|   Fix Found Defects         | [your estimate]     | [your rate]         | "
        "|   Test Fixed Defects        | [your estimate]     | [your rate]         | "
        "| Code Inspection             |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: frontend (forms, UI), backend (APIs, business logic), database integration, recommendation engine, mobile optimization. "
        "Use realistic estimates based on project complexity."
    )
)

testing_engineer = AssistantAgent(
    name="testing_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a testing engineer. You ONLY work on TESTING tasks. "
        "Based on the insurance recommendation app project, estimate the testing work needed. "
        "Provide your response in this EXACT format: "
        "**Testing Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write test plan (TP)        | [your estimate]     | [your rate]         | "
        "| Review TP                   |                     |                     | "
        "|   Preparation for TP        | [your estimate]     | [your rate]         | "
        "|   Review TP Meeting         | [your estimate]     | [your rate]         | "
        "|   Rework                    | [your estimate]     | [your rate]         | "
        "| Execute TP (test cases)     | [your estimate]     | [your rate]         | "
        "| Fix Found Defects           | [your estimate]     | [your rate]         | "
        "Consider: functional testing, UI testing, mobile testing, recommendation accuracy testing, security testing, performance testing. "
        "Use realistic estimates based on project complexity."
    )
)

documentation_engineer = AssistantAgent(
    name="documentation_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a documentation engineer. You ONLY work on DOCUMENTATION tasks. "
        "Based on the insurance recommendation app project, estimate the documentation work needed. "
        "Provide your response in this EXACT format: "
        "**Documentation Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| User Documentation          | [your estimate]     | [your rate]         | "
        "| Review UD                   |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: user manual, help documentation, FAQ, admin documentation, API documentation. "
        "Use realistic estimates based on project complexity."
    )
)

groupchat = GroupChat(
    agents=[
        customer,
        project_manager,
        requirement_engineer,
        system_engineer,
        software_engineer,
        testing_engineer,
        documentation_engineer
    ],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin"
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

customer.initiate_chat(
    manager,
    message=project_manager_message 
)
final_outputs = manager.run()
print(final_outputs)

customer (to chat_manager):


I have received the following message from a customer 
" I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.", 
Now I will talk to each engineers and please everyone give me your estimate plan for this project.

Each engineer must provide estimates in this EXACT format with ASCII table borders:

| Category                    | Amount of work      | Productivity Rate   |
|-----------------------------|---------------------|---------------------|
| [Task name]                 | [Amount]            | [Rate]              |

IMPORTANT: Each engineer must ONLY work on their specific domain and use the exact tab